In [3]:
import pandas as pd
import os
from thyroid.utils.exception import customException

In [8]:
import os

def read_data_file(file_path):
    if file_path.lower().endswith('.data'):
        try:
            with open(file_path, 'r') as file:
                data = file.read()
                return data
        except FileNotFoundError:
            return "File not found."
        except Exception as e:
            return "An error occurred: " + str(e)
    else:
        return "File does not have a .DATA extension."

directory = "raw_data"
file_name = "thyroid0387.data"
file_path = os.path.join(directory, file_name)

result = read_data_file(file_path)
print(result)



File not found.
